In [40]:
import fasttext.util
from sklearn.metrics import classification_report

from utils.common import compress_read

In [41]:
train_df = compress_read('./data/train_shuffle_df.parquet')
val_df = compress_read('./data/val_df.parquet')
test_df = compress_read('./data/test_df.parquet')

In [42]:
with open('./data/ft_train.txt', 'w') as f:
    for i in train_df.to_dict(orient='records'):
        words = str(i['words']).replace(',', ' ')
        label = f'''__label__{i['label']}'''
        f.write(f'''{label}\t{words}\n''')

In [53]:
model = fasttext.train_supervised(
    './data/ft_train.txt',
    label_prefix="__label__",
    lr=0.1,
    epoch=5,
    verbose=2,
    minCount=3,
    wordNgrams=2,
    loss='hs'
)

In [54]:
y_true = []
y_pred = []
with open('./data/ft_val.txt', 'w') as f:
    for i in val_df.sample(frac=1.0).to_dict(orient='records'):
        words = str(i['words']).replace(',', ' ')
        label = f'''__label__{i['label']}'''
        label_hat = model.predict(words, k=1)[0][0]
        # print(label, label_hat)
        y_true.append(label)
        y_pred.append(label_hat)

print(classification_report(y_true, y_pred, digits=4))

              precision    recall  f1-score   support

  __label__0     0.9043    0.8696    0.8866      3574
  __label__1     0.9379    0.8854    0.9109       768
 __label__10     0.9739    0.9740    0.9739     12542
 __label__11     0.9324    0.8912    0.9113       294
 __label__12     0.8831    0.8334    0.8575      2347
 __label__13     0.9301    0.9460    0.9380      8860
  __label__2     0.8973    0.8753    0.8862      1877
  __label__3     0.9201    0.9340    0.9270     14990
  __label__4     0.9014    0.8916    0.8965      3035
  __label__5     0.9391    0.9441    0.9416      3937
  __label__6     0.9297    0.9253    0.9275     15608
  __label__7     0.8836    0.8918    0.8877      4776
  __label__8     0.8783    0.8473    0.8625      1303
  __label__9     0.8902    0.9018    0.8960      6089

    accuracy                         0.9246     80000
   macro avg     0.9144    0.9008    0.9074     80000
weighted avg     0.9245    0.9246    0.9245     80000

